#  Create a Prescription Parser using CRF
This task tests your ability to build a Doctor Prescription Parser with the help of CRF model

Your job is to build a Prescription Parser that takes a prescription (sentence) as an input and find / label the words in that sentence with one of the already pre-defined labels

### Problem: SEQUENCE PREDICTION - Label words in a sentence
#### Input : Doctor Prescription in the form of a sentence split into tokens
- Ex: Take 2 tablets once a day for 10 days

#### Output : FHIR Labels
- ('Take', 'Method')
- ('2', 'Qty') 
- ('tablets', 'Form')
- ('once', 'Frequency')
- ('a', 'Period') 
- ('day', 'PeriodUnit')
- ('for', 'FOR')
- ('10', 'Duration')
- ('days', 'DurationUnit') 

### Major Steps
- Install necessary library
- Import the libraries
- Create training data with labels
    - Split the sentence into tokens
    - Compute POS tags
    - Create triples
- Extract features
- Split the data into training and testing set
- Create CRF model
- Save the CRF model
- Load the CRF model
- Predict on test data
- Accuracy

#### Install necesaary library

#### Import the necessary libraries

In [116]:
import pandas as pd
import textacy 
import spacy
from nltk.tag import pos_tag
#from sklearn.metrics import confusion_matrix, f1_score, classification_report
#from sklearn.pipeline import Pipeline
#from nltk.tag import pos_tag

### Input data (GIVEN)
#### Creating the inputs to the ML model in the following form:
- sigs --> ['take 3 tabs for 10 days']       INPUT SIG
- input_sigs --> [['take', '3', 'tabs', 'for', '10', 'days']]      TOKENS
- output_labels --> [['Method','Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]       LABELS

In [117]:
sigs = ["for 5 to 6 days", "inject 2 units", "x 2 weeks", "x 3 days", "every day", "every 2 weeks", "every 3 days", "every 1 to 2 months", "every 2 to 6 weeks", "every 4 to 6 days", "take two to four tabs", "take 2 to 4 tabs", "take 3 tabs orally bid for 10 days at bedtime", "swallow three capsules tid orally", "take 2 capsules po every 6 hours", "take 2 tabs po for 10 days", "take 100 caps by mouth tid for 10 weeks", "take 2 tabs after an hour", "2 tabs every 4-6 hours", "every 4 to 6 hours", "q46h", "q4-6h", "2 hours before breakfast", "before 30 mins at bedtime", "30 mins before bed", "and 100 tabs twice a month", "100 tabs twice a month", "100 tabs once a month", "100 tabs thrice a month", "3 tabs daily for 3 days then 1 tab per day at bed", "30 tabs 10 days tid", "take 30 tabs for 10 days three times a day", "qid q6h", "bid", "qid", "30 tabs before dinner and bedtime", "30 tabs before dinner & bedtime", "take 3 tabs at bedtime", "30 tabs thrice daily for 10 days ", "30 tabs for 10 days three times a day", "Take 2 tablets a day", "qid for 10 days", "every day", "take 2 caps at bedtime", "apply 3 drops before bedtime", "take three capsules daily", "swallow 3 pills once a day", "swallow three pills thrice a day", "apply daily", "apply three drops before bedtime", "every 6 hours", "before food", "after food", "for 20 days", "for twenty days", "with meals"]
input_sigs = [['for', '5', 'to', '6', 'days'], ['inject', '2', 'units'], ['x', '2', 'weeks'], ['x', '3', 'days'], ['every', 'day'], ['every', '2', 'weeks'], ['every', '3', 'days'], ['every', '1', 'to', '2', 'months'], ['every', '2', 'to', '6', 'weeks'], ['every', '4', 'to', '6', 'days'], ['take', 'two', 'to', 'four', 'tabs'], ['take', '2', 'to', '4', 'tabs'], ['take', '3', 'tabs', 'orally', 'bid', 'for', '10', 'days', 'at', 'bedtime'], ['swallow', 'three', 'capsules', 'tid', 'orally'], ['take', '2', 'capsules', 'po', 'every', '6', 'hours'], ['take', '2', 'tabs', 'po', 'for', '10', 'days'], ['take', '100', 'caps', 'by', 'mouth', 'tid', 'for', '10', 'weeks'], ['take', '2', 'tabs', 'after', 'an', 'hour'], ['2', 'tabs', 'every', '4-6', 'hours'], ['every', '4', 'to', '6', 'hours'], ['q46h'], ['q4-6h'], ['2', 'hours', 'before', 'breakfast'], ['before', '30', 'mins', 'at', 'bedtime'], ['30', 'mins', 'before', 'bed'], ['and', '100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'twice', 'a', 'month'], ['100', 'tabs', 'once', 'a', 'month'], ['100', 'tabs', 'thrice', 'a', 'month'], ['3', 'tabs', 'daily', 'for', '3', 'days', 'then', '1', 'tab', 'per', 'day', 'at', 'bed'], ['30', 'tabs', '10', 'days', 'tid'], ['take', '30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['qid', 'q6h'], ['bid'], ['qid'], ['30', 'tabs', 'before', 'dinner', 'and', 'bedtime'], ['30', 'tabs', 'before', 'dinner', '&', 'bedtime'], ['take', '3', 'tabs', 'at', 'bedtime'], ['30', 'tabs', 'thrice', 'daily', 'for', '10', 'days'], ['30', 'tabs', 'for', '10', 'days', 'three', 'times', 'a', 'day'], ['take', '2', 'tablets', 'a', 'day'], ['qid', 'for', '10', 'days'], ['every', 'day'], ['take', '2', 'caps', 'at', 'bedtime'], ['apply', '3', 'drops', 'before', 'bedtime'], ['take', 'three', 'capsules', 'daily'], ['swallow', '3', 'pills', 'once', 'a', 'day'], ['swallow', 'three', 'pills', 'thrice', 'a', 'day'], ['apply', 'daily'], ['apply', 'three', 'drops', 'before', 'bedtime'], ['every', '6', 'hours'], ['before', 'food'], ['after', 'food'], ['for', '20', 'days'], ['for', 'twenty', 'days'], ['with', 'meals']]
output_labels = [['FOR', 'Duration', 'TO', 'DurationMax', 'DurationUnit'], ['Method', 'Qty', 'Form'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['EVERY', 'Period'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'TO', 'Qty', 'Form'], ['Method', 'Qty', 'Form', 'PO', 'BID', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'TID', 'PO'], ['Method', 'Qty', 'Form', 'PO', 'EVERY', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'PO', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'BY', 'PO', 'TID', 'FOR', 'Duration', 'DurationUnit'], ['Method', 'Qty', 'Form', 'AFTER', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit'], ['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit'], ['Q46H'], ['Q4-6H'], ['Qty', 'PeriodUnit', 'BEFORE', 'WHEN'], ['BEFORE', 'Qty', 'M', 'AT', 'WHEN'], ['Qty', 'M', 'BEFORE', 'WHEN'], ['AND', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Qty', 'Form', 'Frequency', 'FOR', 'Duration', 'DurationUnit', 'THEN', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'AT', 'WHEN'], ['Qty', 'Form', 'Duration', 'DurationUnit', 'TID'], ['Method', 'Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Qty', 'TIMES', 'Period', 'PeriodUnit'], ['QID', 'Q6H'], ['BID'], ['QID'],['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Qty', 'Form', 'BEFORE', 'WHEN', 'AND', 'WHEN'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Qty', 'Form', 'Frequency', 'DAILY', 'FOR', 'Duration', 'DurationUnit'], ['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit', 'Frequency', 'TIMES', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Period', 'PeriodUnit'], ['QID', 'FOR', 'Duration', 'DurationUnit'], ['EVERY', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'AT', 'WHEN'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['Method', 'Qty', 'Form', 'DAILY'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'Qty', 'Form', 'Frequency', 'Period', 'PeriodUnit'], ['Method', 'DAILY'], ['Method', 'Qty', 'Form', 'BEFORE', 'WHEN'], ['EVERY', 'Period', 'PeriodUnit'], ['BEFORE', 'FOOD'], ['AFTER', 'FOOD'], ['FOR', 'Duration', 'DurationUnit'], ['FOR', 'Duration', 'DurationUnit'], ['WITH', 'FOOD']]

In [118]:
len(sigs), len(input_sigs) , len(output_labels)

(56, 56, 56)

### Creating a Tuples Maker method
Create the tuples as given below by writing a function **tuples_maker(input_sigs, output_labels)** and returns **output** as given below

Input(s): 
- input_sigs
- output_lables

Output:

[[('for', 'FOR'),
  ('5', 'Duration'),
  ('to', 'TO'),
  ('6', 'DurationMax'),
  ('days', 'DurationUnit')], [second sentence], ...]

In [119]:
def tuples_maker(inp, out):
    sample_data = []
    for i in range(0,len(inp)):
        imp_out = {}
        imp_out['inp'] = inp[i]
        imp_out['out'] = out[i]
        sample_data.append(imp_out)
    # putting the labels and the words toegther.
    return sample_data

In [120]:
def get_tuples_maker(inp, out):
    tuples= []
    for i in range(0,len(inp)):
        data = tuples_maker(inp[i],out[i])
        tuples.append(data)
    return tuples

In [121]:
data = get_tuples_maker(input_sigs, output_labels)

### Creating the triples_maker( ) for feature extraction
- input: tuples_maker_output
- output: 
[[('for', 'IN', 'FOR'),
  ('5', 'CD', 'Duration'),
  ('to', 'TO', 'TO'),
  ('6', 'CD', 'DurationMax'),
  ('days', 'NNS', 'DurationUnit')], [second sentence], ... ]

In [168]:
def triples_maker(whole_data):
    sample_data = []
    for word in whole_data:
        temp=[]
        tag = pos_tag([word['inp']])
        
        temp.append(word['inp'])
        temp.append(tag[0][1])
        temp.append(word['out'])
        #emp[0] = word['inp']
        #emp[1] = tag[0][1]
        #emp[2] = word['out']
        sample_data.append(temp)
    return sample_data

In [169]:
def get_triples_maker(whole_data):
    triples= []
    for i in whole_data:
        data = triples_maker(i)
        triples.append(data)
    return triples

In [170]:
sample_data = get_triples_maker(data)

In [172]:
sample_data

[[['for', 'IN', 'FOR'],
  ['5', 'CD', 'Duration'],
  ['to', 'TO', 'TO'],
  ['6', 'CD', 'DurationMax'],
  ['days', 'NNS', 'DurationUnit']],
 [['inject', 'NN', 'Method'], ['2', 'CD', 'Qty'], ['units', 'NNS', 'Form']],
 [['x', 'NN', 'FOR'],
  ['2', 'CD', 'Duration'],
  ['weeks', 'NNS', 'DurationUnit']],
 [['x', 'NN', 'FOR'],
  ['3', 'CD', 'Duration'],
  ['days', 'NNS', 'DurationUnit']],
 [['every', 'DT', 'EVERY'], ['day', 'NN', 'Period']],
 [['every', 'DT', 'EVERY'],
  ['2', 'CD', 'Period'],
  ['weeks', 'NNS', 'PeriodUnit']],
 [['every', 'DT', 'EVERY'],
  ['3', 'CD', 'Period'],
  ['days', 'NNS', 'PeriodUnit']],
 [['every', 'DT', 'EVERY'],
  ['1', 'CD', 'Period'],
  ['to', 'TO', 'TO'],
  ['2', 'CD', 'PeriodMax'],
  ['months', 'NNS', 'PeriodUnit']],
 [['every', 'DT', 'EVERY'],
  ['2', 'CD', 'Period'],
  ['to', 'TO', 'TO'],
  ['6', 'CD', 'PeriodMax'],
  ['weeks', 'NNS', 'PeriodUnit']],
 [['every', 'DT', 'EVERY'],
  ['4', 'CD', 'Period'],
  ['to', 'TO', 'TO'],
  ['6', 'CD', 'PeriodMax'],
  ['

### Creating the features extractor method (GIVEN as a BASELINE)
#### The features used are:
- SOS, EOS, lowercase, uppercase, title, digit, postag, previous_tag, next_tag
#### Feel free to include more features

In [173]:
def token_to_features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

### Running the feature extractor on the training data 
- Feature extraction
- Train-test-split

In [174]:
sample_data

[[['for', 'IN', 'FOR'],
  ['5', 'CD', 'Duration'],
  ['to', 'TO', 'TO'],
  ['6', 'CD', 'DurationMax'],
  ['days', 'NNS', 'DurationUnit']],
 [['inject', 'NN', 'Method'], ['2', 'CD', 'Qty'], ['units', 'NNS', 'Form']],
 [['x', 'NN', 'FOR'],
  ['2', 'CD', 'Duration'],
  ['weeks', 'NNS', 'DurationUnit']],
 [['x', 'NN', 'FOR'],
  ['3', 'CD', 'Duration'],
  ['days', 'NNS', 'DurationUnit']],
 [['every', 'DT', 'EVERY'], ['day', 'NN', 'Period']],
 [['every', 'DT', 'EVERY'],
  ['2', 'CD', 'Period'],
  ['weeks', 'NNS', 'PeriodUnit']],
 [['every', 'DT', 'EVERY'],
  ['3', 'CD', 'Period'],
  ['days', 'NNS', 'PeriodUnit']],
 [['every', 'DT', 'EVERY'],
  ['1', 'CD', 'Period'],
  ['to', 'TO', 'TO'],
  ['2', 'CD', 'PeriodMax'],
  ['months', 'NNS', 'PeriodUnit']],
 [['every', 'DT', 'EVERY'],
  ['2', 'CD', 'Period'],
  ['to', 'TO', 'TO'],
  ['6', 'CD', 'PeriodMax'],
  ['weeks', 'NNS', 'PeriodUnit']],
 [['every', 'DT', 'EVERY'],
  ['4', 'CD', 'Period'],
  ['to', 'TO', 'TO'],
  ['6', 'CD', 'PeriodMax'],
  ['

In [176]:
from sklearn.model_selection import train_test_split

def get_features(doc):
    return [token_to_features(doc, i) for i in range (len(doc))]
def get_labels(doc):
    return [label for (token,postag,label) in doc]

X = [get_features(doc) for doc in sample_data]
y = [get_labels(doc) for doc in sample_data]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)


### Training the CRF model with the features extracted using the feature extractor method

In [219]:
from sklearn_crfsuite import CRF, metrics

# Submit training data to the trainer
# Set the parameters of the model
crf = CRF(algorithm='lbfgs',c1=0.1,c2=10,max_iterations=50, keep_tempfiles='task1.txt')
crf.fit(X_train,y_train)
# Providing a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished


CRF(algorithm='lbfgs', c1=0.1, c2=10, keep_tempfiles=None, max_iterations=50)

### Predicting the test data with the built model

In [191]:
labels = crf.classes_
labels

['Q4-6H',
 'Method',
 'Qty',
 'Form',
 'BEFORE',
 'WHEN',
 'AND',
 'BID',
 'EVERY',
 'Period',
 'TO',
 'PeriodMax',
 'PeriodUnit',
 'Frequency',
 'FOR',
 'Duration',
 'DurationUnit',
 'THEN',
 'AT',
 'M',
 'QID',
 'Q6H',
 'PO',
 'BY',
 'TID',
 'DAILY',
 'AFTER',
 'TIMES',
 'FOOD',
 'WITH',
 'Q46H']

In [192]:
sorted_labels = sorted(labels, key=lambda name: (name[1:], name[0]))
sorted_labels

['M',
 'Q4-6H',
 'Q46H',
 'Q6H',
 'DAILY',
 'BEFORE',
 'AFTER',
 'THEN',
 'WHEN',
 'BID',
 'QID',
 'TID',
 'TIMES',
 'WITH',
 'AND',
 'PO',
 'TO',
 'FOOD',
 'FOR',
 'AT',
 'EVERY',
 'BY',
 'Period',
 'PeriodMax',
 'PeriodUnit',
 'Method',
 'Form',
 'Frequency',
 'Qty',
 'Duration',
 'DurationUnit']

In [193]:
y_preds= crf.predict(X_test)

In [194]:
f1_score= metrics.flat_f1_score(y_test,y_preds,average='weighted' , labels=labels)
f1_score

C:\Users\Sarah\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


0.7254317807562923

In [203]:
test = crf.predict("2")
test

[['Q4-6H']]

### Putting all the prediction logic inside a predict method

In [197]:
def predict(sig):
    """
    predict(sig)
    Purpose: Labels the given sig into corresponding labels
    @param sig. A Sentence  # A medical prescription sig written by a doctor
    @return     A list      # A list with predicted labels (first level of labeling)
    >>> predict('2 tabs every 4 hours')
    [['Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit']]
    >>> predict('2 tabs with food')
    [['Qty', 'Form', 'WITH', 'FOOD']]
    >>> predict('2 tabs qid x 30 days')
    [['Qty', 'Form', 'QID', 'FOR', 'Duration', 'DurationUnit']]
    """
    predictions = crf.predict(sig)
    print(predictions)
    return predictions

### Sample predictions

In [198]:
predictions = predict("take 2 tabs every 6 hours x 10 days")

[['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H']]


In [199]:
predictions = predict("2 capsu for 10 day at bed")

[['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H']]


In [204]:
predictions = predict("2 capsu for 10 days at bed")

[['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H']]


In [205]:
predictions = predict("5 days 2 tabs at bed")

[['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H']]


In [206]:
predictions = predict("3 tabs qid x 10 weeks")

[['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H']]


In [207]:
predictions = predict("x 30 days")

[['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H']]


In [208]:
predictions = predict("x 20 months")

[['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H']]


In [209]:
predictions = predict("take 2 tabs po tid for 10 days")

[['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H']]


In [210]:
predictions = predict("take 2 capsules po every 6 hours")

[['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H']]


In [211]:
predictions = predict("inject 2 units pu tid")

[['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H']]


In [212]:
predictions = predict("swallow 3 caps tid by mouth")

[['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H']]


In [213]:
predictions = predict("inject 3 units orally")

[['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H']]


In [214]:
predictions = predict("orally take 3 tabs tid")

[['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H']]


In [215]:
predictions = predict("by mouth take three caps")

[['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H']]


In [216]:
predictions = predict("take 3 tabs orally three times a day for 10 days at bedtime")

[['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H'], ['Q4-6H']]


In [32]:
predictions = predict("take 3 tabs orally bid for 10 days at bedtime")

take 3 tabs orally bid for 10 days at bedtime
[['Method', 'Qty', 'Form', 'PO', 'BID', 'FOR', 'Duration', 'DurationUnit', 'AT', 'WHEN']]


In [33]:
predictions = predict("take 3 tabs bid orally at bed")

take 3 tabs bid orally at bed
[['Method', 'Qty', 'Form', 'BID', 'PO', 'AT', 'WHEN']]


In [34]:
predictions = predict("take 10 capsules by mouth qid")

take 10 capsules by mouth qid
[['Method', 'Qty', 'Form', 'BY', 'PO', 'QID']]


In [35]:
predictions = predict("inject 10 units orally qid x 3 months")

inject 10 units orally qid x 3 months
[['Method', 'Qty', 'Form', 'PO', 'QID', 'FOR', 'Duration', 'DurationUnit']]


In [36]:
prediction = predict("please take 2 tablets per day for a month in the morning and evening each day")

please take 2 tablets per day for a month in the morning and evening each day
[['Method', 'Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'FOR', 'Period', 'PeriodUnit', 'EVERY', 'Period', 'PeriodUnit', 'AND', 'EVERY', 'Period', 'PeriodUnit']]


In [37]:
prediction = predict("Amoxcicillin QID 30 tablets")

Amoxcicillin QID 30 tablets
[['WHEN', 'QID', 'Qty', 'Form']]


In [38]:
prediction = predict("take 3 tabs TID for 90 days with food")

take 3 tabs TID for 90 days with food
[['Method', 'Qty', 'Form', 'TID', 'FOR', 'Duration', 'DurationUnit', 'WITH', 'FOOD']]


In [39]:
prediction = predict("with food take 3 tablets per day for 90 days")

with food take 3 tablets per day for 90 days
[['WITH', 'FOOD', 'Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']]


In [40]:
prediction = predict("with food take 3 tablets per week for 90 weeks")

with food take 3 tablets per week for 90 weeks
[['WITH', 'FOOD', 'Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'FOR', 'Duration', 'DurationUnit']]


In [41]:
prediction = predict("take 2-4 tabs")

take 2-4 tabs
[['Method', 'Qty', 'Form']]


In [42]:
prediction = predict("take 2 to 4 tabs")

take 2 to 4 tabs
[['Method', 'Qty', 'TO', 'Qty', 'Form']]


In [43]:
prediction = predict("take two to four tabs")

take two to four tabs
[['Method', 'Qty', 'TO', 'Qty', 'Form']]


In [44]:
prediction = predict("take 2-4 tabs for 8 to 9 days")

take 2-4 tabs for 8 to 9 days
[['Method', 'Qty', 'Form', 'FOR', 'Duration', 'TO', 'PeriodMax', 'PeriodUnit']]


In [45]:
prediction = predict("take 20 tabs every 6 to 8 days")

take 20 tabs every 6 to 8 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [46]:
prediction = predict("take 2 tabs every 4 to 6 days")

take 2 tabs every 4 to 6 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [47]:
prediction = predict("take 2 tabs every 2 to 10 weeks")

take 2 tabs every 2 to 10 weeks
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [48]:
prediction = predict("take 2 tabs every 4 to 6 days")

take 2 tabs every 4 to 6 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [49]:
prediction = predict("take 2 tabs every 2 to 10 months")

take 2 tabs every 2 to 10 months
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [50]:
prediction = predict("every 60 mins")

every 60 mins
[['EVERY', 'Period', 'PeriodUnit']]


In [51]:
prediction = predict("every 10 mins")

every 10 mins
[['EVERY', 'Period', 'PeriodUnit']]


In [52]:
prediction = predict("every two to four months")

every two to four months
[['EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [53]:
prediction = predict("take 2 tabs every 3 to 4 days")

take 2 tabs every 3 to 4 days
[['Method', 'Qty', 'Form', 'EVERY', 'Period', 'TO', 'PeriodMax', 'PeriodUnit']]


In [54]:
prediction = predict("every 3 to 4 days take 20 tabs")

every 3 to 4 days take 20 tabs
[['EVERY', 'Period', 'TO', 'DurationMax', 'DurationUnit', 'Method', 'Qty', 'Form']]


In [55]:
prediction = predict("once in every 3 days take 3 tabs")

once in every 3 days take 3 tabs
[['FOR', 'Duration', 'EVERY', 'Period', 'PeriodUnit', 'Method', 'Qty', 'Form']]


In [56]:
prediction = predict("take 3 tabs once in every 3 days")

take 3 tabs once in every 3 days
[['Method', 'Qty', 'Form', 'Frequency', 'PeriodUnit', 'EVERY', 'Period', 'PeriodUnit']]


In [57]:
prediction = predict("orally take 20 tabs every 4-6 weeks")

orally take 20 tabs every 4-6 weeks
[['PO', 'Method', 'Qty', 'Form', 'EVERY', 'Period', 'PeriodUnit']]


In [58]:
prediction = predict("10 tabs x 2 days")

10 tabs x 2 days
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [59]:
prediction = predict("3 capsule x 15 days")

3 capsule x 15 days
[['Qty', 'Form', 'FOR', 'Duration', 'DurationUnit']]


In [60]:
prediction = predict("10 tabs")

10 tabs
[['Qty', 'Form']]
